In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from copy import deepcopy
import numpy as np
import statsmodels.api as sm
import statistics as sts
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import typing

In [2]:
tqdm.pandas()

In [3]:
df = pd.read_csv('train.csv')

D:\programmes\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## NaN preprocessing

In [4]:
df.isnull().sum().iloc[np.where(df.isnull().sum() != 0)]

floor                             176237
osm_city_nearest_population           55
reform_house_population_1000       14596
reform_house_population_500        27234
reform_mean_floor_count_1000       16708
reform_mean_floor_count_500        30168
reform_mean_year_building_1000     16239
reform_mean_year_building_500      29637
street                              1606
dtype: int64

In [9]:
class NanFighter:
    
    @staticmethod
    def floor_fighter(df):
    
        def determine_hight_category(x):
            '''
            1 - floors from 1 to 5
            2 - floors from 6 to 10
            3 - floors from 10 to 30
            4 - floors from 30 to 90
            '''
            try:
                x = float(x)
            except:
                return 1
            else:
                if 1 <= x < 5:
                    return 1
                elif 5 <= x < 10:
                    return 2
                elif 10 <= x < 30:
                    return 3
                else:
                    return 4

        def is_wholesale(x):
            x = str(x)
            if len(x.split(',')) >= 2:
                return 1
            return 0
        
        df['wholesale'] = df['floor'].apply(is_wholesale)
        df['floor_hight_category'] = df['floor'].apply(determine_hight_category)
        df.drop(['floor'], inplace=True, axis=1)
        return df
    
    @staticmethod
    def osm_city_nearest_population_fighter(df):
        
        def determine_osm_city_nearest_population(observation):
            if pd.isnull(observation.osm_city_nearest_population):
                value = aggregation_osm_city_nearest_population.loc[observation.region]
                observation.osm_city_nearest_population = value
            return observation.osm_city_nearest_population
        
        aggregation_osm_city_nearest_population = df.groupby(by=['region']). \
                                          aggregate({'osm_city_nearest_population': np.mean}). \
                                          osm_city_nearest_population
        
        df['osm_city_nearest_population'] = df[['osm_city_nearest_population', 'region']]. \
                                    apply(determine_osm_city_nearest_population, axis=1)
        
        return df
    
    @staticmethod
    def reform_house_population_500_fighter(df):
        aggregation_reform_house_population = df.groupby(by=['city']).aggregate({'reform_house_population_500': np.mean})
        data_for_reform_house_population_500 = aggregation_reform_house_population.reform_house_population_500
        data_for_reform_house_population_500_mode = sts.mode(data_for_reform_house_population_500)
        
        def determine_reform_house_population_500(observation):
            if pd.isnull(observation.reform_house_population_500):
                value = data_for_reform_house_population_500.loc[observation.city]
                if pd.isnull(value):
                    value = data_for_reform_house_population_500_mode
                observation.reform_house_population_500 = value
            return observation.reform_house_population_500
        
        df['reform_house_population_500'] = df[['city', 'reform_house_population_500']]. \
                                    apply(determine_reform_house_population_500, axis=1)
        return df
    
    @staticmethod
    def reform_house_population_1000_fighter(df):
        aggregation_reform_house_population = df.groupby(by=['city']).aggregate({'reform_house_population_1000': np.mean})
        data_for_reform_house_population_1000 = aggregation_reform_house_population.reform_house_population_1000
        data_for_reform_house_population_1000_mode = sts.mode(data_for_reform_house_population_1000)
        
        def determine_reform_house_population_1000(observation):
            if pd.isnull(observation.reform_house_population_1000):
                value = data_for_reform_house_population_1000.loc[observation.city]
                if pd.isnull(value):
                    value = data_for_reform_house_population_1000_mode
                observation.reform_house_population_1000 = value
            return observation.reform_house_population_1000
        df['reform_house_population_1000'] = df[['city', 'reform_house_population_1000']].apply(determine_reform_house_population_1000, axis=1)
        return df
    
    @staticmethod
    def reform_mean_floor_count_500_fighter(df):
        aggregation_reform_mean_floor_count = df.groupby(by=['city']).aggregate({'reform_mean_floor_count_500': np.mean})
        data_for_reform_mean_floor_count_500 = aggregation_reform_mean_floor_count.reform_mean_floor_count_500
        data_for_reform_mean_floor_count_500_mode = sts.mode(data_for_reform_mean_floor_count_500)
        
        def determine_mean_floor_count_500(observation):
            if pd.isnull(observation.reform_mean_floor_count_500):
                value = data_for_reform_mean_floor_count_500.loc[observation.city]
                if pd.isnull(value):
                    value = data_for_reform_mean_floor_count_500_mode
                observation.reform_mean_floor_count_500 = value
            return observation.reform_mean_floor_count_500
        df['reform_mean_floor_count_500'] = df[['city', 'reform_mean_floor_count_500']]. \
                                    apply(determine_mean_floor_count_500, axis=1)
        return df
        
    @staticmethod  
    def reform_mean_floor_count_1000_fighter(df):
        aggregation_reform_mean_floor_count = df.groupby(by=['city']).aggregate({'reform_mean_floor_count_1000': np.mean})
        data_for_reform_mean_floor_count_1000 = aggregation_reform_mean_floor_count.reform_mean_floor_count_1000
        data_for_reform_mean_floor_count_1000_mode = sts.mode(data_for_reform_mean_floor_count_1000)
    
        def determine_mean_floor_count_1000(observation):
            if pd.isnull(observation.reform_mean_floor_count_1000):
                value = data_for_reform_mean_floor_count_1000.loc[observation.city]
                if pd.isnull(value):
                    value = data_for_reform_mean_floor_count_1000_mode
                observation.reform_mean_floor_count_1000 = value
            return observation.reform_mean_floor_count_1000
        
        df['reform_mean_floor_count_1000'] = df[['city', 'reform_mean_floor_count_1000']]. \
                                     apply(determine_mean_floor_count_1000, axis=1)
        return df
    
    @staticmethod
    def street_fighter(df):
        def fill_street(x):
            if x.count() <= 0:
                return np.nan
            return x.value_counts().index[0]

        df['street'] = df.groupby('city')['street'].transform(fill_street)
        df['street'] = df['street'].fillna(df['street'].value_counts().idxmax())
        return df
    
    @staticmethod
    def reform_mean_year_building_500_fighter(df):
        aggregation_year_mean_street = df.groupby(by=['street']).aggregate({'reform_mean_year_building_500': np.mean})
        aggregation_year_mean_city = df.groupby(by=['city']).aggregate({'reform_mean_year_building_500': np.mean})
        aggregation_year_mean_region = df.groupby(by=['region']).aggregate({'reform_mean_year_building_500': np.mean})
        data_for_year_mean_street_500 = aggregation_year_mean_street.reform_mean_year_building_500
        data_for_year_mean_city_500 = aggregation_year_mean_city.reform_mean_year_building_500
        data_for_year_mean_region_500 = aggregation_year_mean_region.reform_mean_year_building_500
        data_for_year_mean_street_500_mode = sts.mode(data_for_year_mean_region_500)
        
        def determine_mean_year_500(observation):
            if pd.isnull(observation.reform_mean_year_building_500):
                street, city, region = observation.street, observation.city, observation.region
                value = data_for_year_mean_city_500.loc[city]
                if pd.isnull(value):
                    value = data_for_year_mean_region_500.loc[region]
                    if pd.isnull(value):
                        value = data_for_year_mean_street_500_mode
                observation.reform_mean_year_building_500 = value
            return observation.reform_mean_year_building_500
        df['reform_mean_year_building_500'] = df[['street', 'city', 'region', 'reform_mean_year_building_500']]. \
                                      apply(determine_mean_year_500, axis=1)
        return df
    
    @staticmethod
    def reform_mean_year_building_1000_fighter(df):
        aggregation_year_mean_street = df.groupby(by=['street']).aggregate({'reform_mean_year_building_1000': np.mean})
        aggregation_year_mean_city = df.groupby(by=['city']).aggregate({'reform_mean_year_building_1000': np.mean})
        aggregation_year_mean_region = df.groupby(by=['region']).aggregate({'reform_mean_year_building_1000': np.mean})
        data_for_year_mean_street_1000 = aggregation_year_mean_street.reform_mean_year_building_1000
        data_for_year_mean_city_1000 = aggregation_year_mean_city.reform_mean_year_building_1000
        data_for_year_mean_region_1000 = aggregation_year_mean_region.reform_mean_year_building_1000
        data_for_year_mean_street_1000_mode = sts.mode(data_for_year_mean_region_1000)
        
        def determine_mean_year_1000(observation):
            if pd.isnull(observation.reform_mean_year_building_1000):
                street, city, region = observation.street, observation.city, observation.region
                value = data_for_year_mean_city_1000.loc[city]
                if pd.isnull(value):
                    value = data_for_year_mean_region_1000.loc[region]
                    if pd.isnull(value):
                        value = data_for_year_mean_street_1000_mode
                observation.reform_mean_year_building_1000 = value
            return observation.reform_mean_year_building_1000
        
        df['reform_mean_year_building_1000'] = df[['street', 'city', 'region', 'reform_mean_year_building_1000']]. \
                                       apply(determine_mean_year_1000, axis=1)
        return df
    
    @staticmethod
    def fight(df):
        result_df = deepcopy(df)
        for feature in df.columns:
            fighter_name = feature + "_fighter"
            fighter = getattr(NanFighter, fighter_name, None)
            if fighter:
                result_df = fighter(result_df)
        return result_df

In [10]:
df = NanFighter.fight(df)

In [30]:
import scipy
scipy.__version__

'1.5.2'

In [ ]:
from sklearn.preprocessing import LabelEncoder

categorial_features = [
    'osm_city_nearest_name',
    'region']

def labencoder(df, cols):
    for i in cols:
        labelencoder = LabelEncoder()
        df["cat" + i] = labelencoder.fit_transform(df[i])
    return df
lab_encoder(df, categorial_features)

#### Check for nulls

In [8]:
df.isnull().sum().iloc[np.where(df.isnull().sum() != 0)]

Series([], dtype: int64)

## Feature Selection

### Stepwize selection

In [17]:
target = 'per_square_meter_price'

categorial_features = [
    'city',
    'floor',
    'id',
    'osm_city_nearest_name',
    'region',
    'street',
    'realty_type',
    'price_type'
]

drop_columns = categorial_features + ['date', 'per_square_meter_price']

x_columns = list(set(df.columns) - set(drop_columns))

In [18]:
def stepwize_regression(df):
    
    df_temp = deepcopy(df)
    p_value_threshold = 0.05  # if greater, than should be dropped
    
    current_columns = []
    for column in x_columns:
        current_columns.append(column)
        X = df_temp[current_columns].astype(float)
        y = list(df_temp[target])
        results = sm.OLS(y, X).fit()
        appropriate_features = results.pvalues.iloc[np.where(results.pvalues <= p_value_threshold)]
    return list(appropriate_features.index)

In [19]:
appropriate_features = stepwize_regression(df)

# appropriate_features = []
# with open('features_after_stepwize.txt', 'r') as file:
#     for feature in file:
#         appropriate_features.append(feature.strip())

In [23]:
import statistics

In [27]:
!pip freeze

alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.10.0
anaconda-project==0.8.3
argh==0.26.2
argon2-cffi @ file:///C:/ci/argon2-cffi_1596828585465/work
asn1crypto @ file:///tmp/build/80754af9/asn1crypto_1596577642040/work
astroid @ file:///C:/ci/astroid_1592487315634/work
astropy==4.0.2
async-generator==1.10
atomicwrites==1.4.0
attrs @ file:///tmp/build/80754af9/attrs_1604765588209/work
autopep8 @ file:///tmp/build/80754af9/autopep8_1596578164842/work
Babel @ file:///tmp/build/80754af9/babel_1605108370292/work
backcall==0.2.0
backports.functools-lru-cache==1.6.1
backports.shutil-get-terminal-size==1.0.0
backports.tempfile==1.0
backports.weakref==1.0.post1
bcrypt @ file:///C:/ci/bcrypt_1597936263757/work
beautifulsoup4 @ file:///home/conda/feedstock_root/build_artifacts/beautifulsoup4_1601745390275/work
bitarray @ file:///C:/ci/bitarray_1605065210072/work
bkcharts==0.2
bleach @ file:///tmp/build/80754af9/bleach_1600439572647/work
bokeh @ file:///C:/ci/bokeh_1603297934731/wor

# FITTING

## Submission dataset

In [11]:
df_test = pd.read_csv('test.csv')

In [12]:
df_test = NanFighter.fight(df_test)

In [14]:
df_submission = df_test[appropriate_features]

NameError: name 'appropriate_features' is not defined

In [102]:
df_submission.isnull().sum().iloc[np.where(df_submission.isnull().sum() != 0)]

Series([], dtype: int64)

In [41]:
X = df[appropriate_features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [42]:
model = LGBMRegressor()

In [59]:
def use_model(model, parameters={}):
    model = model(**parameters)
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    return y_hat, model

In [55]:
parameters  = {
    'eval_metric': CatBoostEvalMetricDeviation()
}

In [56]:
class CatBoostEvalMetricDeviation(object):
    
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        print(approxes)
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        preds = np.array(approxes[0])
        target = np.array(target)
        return deviation_metric(target, preds), 0

### Metric

In [57]:
THRESHOLD = 0.15
NEGATIVE_WEIGHT = 1.1

def deviation_metric_one_sample(y_true: typing.Union[float, int], y_pred: typing.Union[float, int]) -> float:
    """
    Реализация кастомной метрики для хакатона.

    :param y_true: float, реальная цена
    :param y_pred: float, предсказанная цена
    :return: float, значение метрики
    """
    deviation = (y_pred - y_true) / np.maximum(1e-8, y_true)
    if np.abs(deviation) <= THRESHOLD:
        return 0
    elif deviation <= - 4 * THRESHOLD:
        return 9 * NEGATIVE_WEIGHT
    elif deviation < -THRESHOLD:
        return NEGATIVE_WEIGHT * ((deviation / THRESHOLD) + 1) ** 2
    elif deviation < 4 * THRESHOLD:
        return ((deviation / THRESHOLD) - 1) ** 2
    else:
        return 9


def deviation_metric(y_true: np.array, y_pred: np.array) -> float:
    return np.array([deviation_metric_one_sample(y_true[n], y_pred[n]) for n in range(len(y_true))]).mean()

In [58]:
deviation_metric(y_test.reset_index(drop=True), use_model(CatBoostRegressor, parameters))

Learning rate set to 0.099854
0:	learn: 6.4112672	total: 1.05s	remaining: 17m 32s
1:	learn: 6.1850118	total: 2.12s	remaining: 17m 36s
2:	learn: 5.9934361	total: 3.09s	remaining: 17m 7s
3:	learn: 5.8307361	total: 4.02s	remaining: 16m 40s
4:	learn: 5.6794168	total: 5.04s	remaining: 16m 42s
5:	learn: 5.5640229	total: 6.16s	remaining: 17m
6:	learn: 5.4700972	total: 7.29s	remaining: 17m 13s
7:	learn: 5.3775436	total: 8.21s	remaining: 16m 58s
8:	learn: 5.2772167	total: 9.26s	remaining: 16m 59s
9:	learn: 5.1918529	total: 10.3s	remaining: 16m 57s
10:	learn: 5.1079764	total: 11.2s	remaining: 16m 49s
11:	learn: 5.0288299	total: 12.2s	remaining: 16m 42s
12:	learn: 4.9609429	total: 13.1s	remaining: 16m 37s
13:	learn: 4.8987823	total: 14.1s	remaining: 16m 33s
14:	learn: 4.8336140	total: 15.1s	remaining: 16m 34s
15:	learn: 4.7814090	total: 16.1s	remaining: 16m 29s
16:	learn: 4.7432440	total: 17.1s	remaining: 16m 26s
17:	learn: 4.7070804	total: 18.1s	remaining: 16m 27s
18:	learn: 4.6699510	total: 19.

70:	learn: 4.1994356	total: 1m 15s	remaining: 16m 29s
71:	learn: 4.1983803	total: 1m 16s	remaining: 16m 27s
72:	learn: 4.1972899	total: 1m 18s	remaining: 16m 30s
73:	learn: 4.1922159	total: 1m 19s	remaining: 16m 30s
74:	learn: 4.1907485	total: 1m 20s	remaining: 16m 36s
75:	learn: 4.1899194	total: 1m 22s	remaining: 16m 38s
76:	learn: 4.1886098	total: 1m 23s	remaining: 16m 38s
77:	learn: 4.1865718	total: 1m 24s	remaining: 16m 42s
78:	learn: 4.1845320	total: 1m 25s	remaining: 16m 41s
79:	learn: 4.1824707	total: 1m 27s	remaining: 16m 42s
80:	learn: 4.1741786	total: 1m 28s	remaining: 16m 42s
81:	learn: 4.1725551	total: 1m 29s	remaining: 16m 40s
82:	learn: 4.1693364	total: 1m 30s	remaining: 16m 37s
83:	learn: 4.1675054	total: 1m 31s	remaining: 16m 36s
84:	learn: 4.1659590	total: 1m 32s	remaining: 16m 35s
85:	learn: 4.1653295	total: 1m 33s	remaining: 16m 38s
86:	learn: 4.1637814	total: 1m 35s	remaining: 16m 44s
87:	learn: 4.1626124	total: 1m 36s	remaining: 16m 44s
88:	learn: 4.1624359	total: 

140:	learn: 4.0982416	total: 2m 31s	remaining: 15m 21s
141:	learn: 4.0972805	total: 2m 32s	remaining: 15m 19s
142:	learn: 4.0959469	total: 2m 33s	remaining: 15m 17s
143:	learn: 4.0961286	total: 2m 34s	remaining: 15m 16s
144:	learn: 4.0954816	total: 2m 35s	remaining: 15m 14s
145:	learn: 4.0951749	total: 2m 36s	remaining: 15m 13s
146:	learn: 4.0939583	total: 2m 37s	remaining: 15m 11s
147:	learn: 4.0938438	total: 2m 38s	remaining: 15m 9s
148:	learn: 4.0928369	total: 2m 38s	remaining: 15m 7s
149:	learn: 4.0916423	total: 2m 39s	remaining: 15m 6s
150:	learn: 4.0810619	total: 2m 40s	remaining: 15m 4s
151:	learn: 4.0793630	total: 2m 41s	remaining: 15m 2s
152:	learn: 4.0779579	total: 2m 42s	remaining: 15m 1s
153:	learn: 4.0778276	total: 2m 43s	remaining: 14m 59s
154:	learn: 4.0777743	total: 2m 44s	remaining: 14m 57s
155:	learn: 4.0770636	total: 2m 45s	remaining: 14m 56s
156:	learn: 4.0761172	total: 2m 46s	remaining: 14m 54s
157:	learn: 4.0751290	total: 2m 47s	remaining: 14m 52s
158:	learn: 4.07

210:	learn: 4.0247448	total: 3m 39s	remaining: 13m 38s
211:	learn: 4.0229403	total: 3m 39s	remaining: 13m 37s
212:	learn: 4.0223806	total: 3m 40s	remaining: 13m 36s
213:	learn: 4.0221689	total: 3m 41s	remaining: 13m 35s
214:	learn: 4.0214744	total: 3m 42s	remaining: 13m 34s
215:	learn: 4.0203792	total: 3m 43s	remaining: 13m 32s
216:	learn: 4.0208445	total: 3m 44s	remaining: 13m 31s
217:	learn: 4.0208544	total: 3m 45s	remaining: 13m 30s
218:	learn: 4.0202297	total: 3m 46s	remaining: 13m 28s
219:	learn: 4.0164148	total: 3m 47s	remaining: 13m 27s
220:	learn: 4.0151868	total: 3m 48s	remaining: 13m 26s
221:	learn: 4.0145448	total: 3m 49s	remaining: 13m 24s
222:	learn: 4.0142754	total: 3m 50s	remaining: 13m 23s
223:	learn: 4.0134717	total: 3m 51s	remaining: 13m 22s
224:	learn: 4.0114566	total: 3m 52s	remaining: 13m 21s
225:	learn: 4.0107879	total: 3m 53s	remaining: 13m 20s
226:	learn: 4.0102809	total: 3m 54s	remaining: 13m 19s
227:	learn: 4.0094853	total: 3m 55s	remaining: 13m 18s
228:	learn

280:	learn: 3.9665644	total: 4m 47s	remaining: 12m 14s
281:	learn: 3.9664105	total: 4m 48s	remaining: 12m 13s
282:	learn: 3.9662749	total: 4m 49s	remaining: 12m 12s
283:	learn: 3.9656549	total: 4m 50s	remaining: 12m 11s
284:	learn: 3.9650454	total: 4m 51s	remaining: 12m 10s
285:	learn: 3.9649286	total: 4m 52s	remaining: 12m 9s
286:	learn: 3.9645159	total: 4m 52s	remaining: 12m 7s
287:	learn: 3.9645767	total: 4m 53s	remaining: 12m 6s
288:	learn: 3.9638665	total: 4m 54s	remaining: 12m 5s
289:	learn: 3.9625484	total: 4m 55s	remaining: 12m 4s
290:	learn: 3.9622081	total: 4m 56s	remaining: 12m 3s
291:	learn: 3.9618656	total: 4m 57s	remaining: 12m 2s
292:	learn: 3.9590422	total: 4m 59s	remaining: 12m 1s
293:	learn: 3.9571290	total: 5m	remaining: 12m
294:	learn: 3.9571338	total: 5m	remaining: 11m 59s
295:	learn: 3.9562759	total: 5m 2s	remaining: 11m 58s
296:	learn: 3.9553870	total: 5m 3s	remaining: 11m 57s
297:	learn: 3.9551767	total: 5m 4s	remaining: 11m 57s
298:	learn: 3.9534707	total: 5m 5

...

772:	learn: 3.7548011	total: 12m 47s	remaining: 3m 45s
773:	learn: 3.7545613	total: 12m 48s	remaining: 3m 44s
774:	learn: 3.7545232	total: 12m 49s	remaining: 3m 43s
775:	learn: 3.7528750	total: 12m 50s	remaining: 3m 42s
776:	learn: 3.7526278	total: 12m 51s	remaining: 3m 41s
777:	learn: 3.7526620	total: 12m 52s	remaining: 3m 40s
778:	learn: 3.7522175	total: 12m 53s	remaining: 3m 39s
779:	learn: 3.7519383	total: 12m 54s	remaining: 3m 38s
780:	learn: 3.7516650	total: 12m 55s	remaining: 3m 37s
781:	learn: 3.7510184	total: 12m 55s	remaining: 3m 36s
782:	learn: 3.7511807	total: 12m 56s	remaining: 3m 35s
783:	learn: 3.7510636	total: 12m 57s	remaining: 3m 34s
784:	learn: 3.7508483	total: 12m 58s	remaining: 3m 33s
785:	learn: 3.7508680	total: 12m 59s	remaining: 3m 32s
786:	learn: 3.7493182	total: 13m	remaining: 3m 31s
787:	learn: 3.7487132	total: 13m 1s	remaining: 3m 30s
788:	learn: 3.7485049	total: 13m 2s	remaining: 3m 29s
789:	learn: 3.7485887	total: 13m 3s	remaining: 3m 28s
790:	learn: 3.747

842:	learn: 3.7312775	total: 13m 53s	remaining: 2m 35s
843:	learn: 3.7312480	total: 13m 54s	remaining: 2m 34s
844:	learn: 3.7306632	total: 13m 55s	remaining: 2m 33s
845:	learn: 3.7306141	total: 13m 55s	remaining: 2m 32s
846:	learn: 3.7304919	total: 13m 56s	remaining: 2m 31s
847:	learn: 3.7303301	total: 13m 57s	remaining: 2m 30s
848:	learn: 3.7305494	total: 13m 58s	remaining: 2m 29s
849:	learn: 3.7304100	total: 13m 59s	remaining: 2m 28s
850:	learn: 3.7298458	total: 14m	remaining: 2m 27s
851:	learn: 3.7295215	total: 14m 1s	remaining: 2m 26s
852:	learn: 3.7294214	total: 14m 2s	remaining: 2m 25s
853:	learn: 3.7288748	total: 14m 3s	remaining: 2m 24s
854:	learn: 3.7285653	total: 14m 4s	remaining: 2m 23s
855:	learn: 3.7283780	total: 14m 5s	remaining: 2m 22s
856:	learn: 3.7283530	total: 14m 6s	remaining: 2m 21s
857:	learn: 3.7277636	total: 14m 7s	remaining: 2m 20s
858:	learn: 3.7271021	total: 14m 8s	remaining: 2m 19s
859:	learn: 3.7271339	total: 14m 9s	remaining: 2m 18s
860:	learn: 3.7270893	t

912:	learn: 3.7131790	total: 15m	remaining: 1m 25s
913:	learn: 3.7132187	total: 15m 1s	remaining: 1m 24s
914:	learn: 3.7130105	total: 15m 2s	remaining: 1m 23s
915:	learn: 3.7125384	total: 15m 3s	remaining: 1m 22s
916:	learn: 3.7119749	total: 15m 4s	remaining: 1m 21s
917:	learn: 3.7120945	total: 15m 5s	remaining: 1m 20s
918:	learn: 3.7112257	total: 15m 6s	remaining: 1m 19s
919:	learn: 3.7097761	total: 15m 7s	remaining: 1m 18s
920:	learn: 3.7092280	total: 15m 8s	remaining: 1m 17s
921:	learn: 3.7090835	total: 15m 8s	remaining: 1m 16s
922:	learn: 3.7071972	total: 15m 9s	remaining: 1m 15s
923:	learn: 3.7072045	total: 15m 10s	remaining: 1m 14s
924:	learn: 3.7068748	total: 15m 11s	remaining: 1m 13s
925:	learn: 3.7070007	total: 15m 12s	remaining: 1m 12s
926:	learn: 3.7065937	total: 15m 13s	remaining: 1m 11s
927:	learn: 3.7066312	total: 15m 14s	remaining: 1m 10s
928:	learn: 3.7061396	total: 15m 15s	remaining: 1m 9s
929:	learn: 3.7058673	total: 15m 16s	remaining: 1m 9s
930:	learn: 3.7053165	tota

983:	learn: 3.6911806	total: 16m 15s	remaining: 15.9s
984:	learn: 3.6907947	total: 16m 16s	remaining: 14.9s
985:	learn: 3.6905226	total: 16m 17s	remaining: 13.9s
986:	learn: 3.6899447	total: 16m 18s	remaining: 12.9s
987:	learn: 3.6895710	total: 16m 19s	remaining: 11.9s
988:	learn: 3.6893801	total: 16m 20s	remaining: 10.9s
989:	learn: 3.6893892	total: 16m 21s	remaining: 9.91s
990:	learn: 3.6893558	total: 16m 22s	remaining: 8.92s
991:	learn: 3.6892035	total: 16m 23s	remaining: 7.93s
992:	learn: 3.6887987	total: 16m 24s	remaining: 6.94s
993:	learn: 3.6887206	total: 16m 25s	remaining: 5.95s
994:	learn: 3.6883832	total: 16m 26s	remaining: 4.96s
995:	learn: 3.6882823	total: 16m 27s	remaining: 3.96s
996:	learn: 3.6878763	total: 16m 28s	remaining: 2.97s
997:	learn: 3.6878382	total: 16m 29s	remaining: 1.98s
998:	learn: 3.6877693	total: 16m 30s	remaining: 991ms
999:	learn: 3.6877855	total: 16m 31s	remaining: 0us


3.7414441351728747

In [61]:
y_hat, rfr = use_model(RandomForestRegressor)

In [62]:
deviation_metric(y_test.reset_index(drop=True), y_hat)

2.354308458480482

In [106]:
df_test

,city,id,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,...,region,lat,lng,total_square,street,date,realty_type,price_type,wholesale,floor_hight_category
0,Курск,COL_289284,7,55,85,117,0,0,0,0,...,Курская область,51.709255,36.147908,156.148996,S7137,2020-09-06,100,1,0,1
1,Сургут,COL_289305,8,70,112,140,0,0,0,0,...,Ханты-Мансийский АО,61.233240,73.462509,190.737943,S29120,2020-09-06,110,1,0,1
2,Тюмень,COL_289318,3,28,67,122,0,0,0,0,...,Тюменская область,57.143110,65.554573,457.118051,S10922,2020-09-06,10,1,0,4
3,Иркутск,COL_289354,5,76,139,231,0,0,0,0,...,Иркутская область,52.281380,104.282975,66.503622,S3265,2020-09-06,100,1,0,1
4,Курск,COL_289399,8,105,189,279,0,0,2,9,...,Курская область,51.729706,36.194019,23.864915,S7137,2020-09-06,10,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2969,Красноярск,COL_455089,0,3,6,16,0,0,0,0,...,Красноярский край,55.978180,92.891457,2149.546362,S2336,2020-12-27,100,1,0,1
2970,Томск,COL_455212,2,33,111,222,0,0,1,1,...,Томская область,56.459183,84.979872,359.371061,S11347,2020-12-27,10,1,0,1
2971,Калуга,COL_455261,2,25,54,99,0,0,0,2,...,Калужская область,54.523247,36.295168,87.820453,S10442,2020-12-27,100,1,0,4
2972,Нижний Новгород,COL_455381,13,70,114,158,0,0,0,0,...,Нижегородская область,56.328236,43.990039,208.625735,S21735,2020-12-27,10,1,0,4


In [104]:
y_submission = rfr.predict(df_submission)

In [108]:
def save_submission(id_column, y_hat):
    df_submission = pd.DataFrame({'id':id_column, 'per_square_meter_price': y_hat})
    df_submission.to_csv('test_submission.csv', index=False, sep=',')

In [109]:
save_submission(df_test['id'], y_submission)

In [112]:
appropriate_features

['osm_shops_points_in_0.005',
 'osm_leisure_points_in_0.01',
 'osm_catering_points_in_0.005',
 'osm_hotels_points_in_0.005',
 'osm_culture_points_in_0.005',
 'osm_crossing_points_in_0.001',
 'osm_train_stop_points_in_0.0075',
 'osm_catering_points_in_0.0075',
 'osm_catering_points_in_0.01',
 'osm_transport_stop_points_in_0.0075',
 'osm_hotels_points_in_0.01',
 'osm_historic_points_in_0.0075',
 'osm_leisure_points_in_0.0075',
 'osm_healthcare_points_in_0.01',
 'osm_culture_points_in_0.001',
 'reform_mean_floor_count_500',
 'osm_amenity_points_in_0.001',
 'osm_building_points_in_0.005',
 'lng',
 'osm_culture_points_in_0.01',
 'reform_house_population_1000',
 'osm_crossing_points_in_0.005',
 'osm_healthcare_points_in_0.0075',
 'reform_mean_year_building_1000',
 'reform_mean_floor_count_1000',
 'osm_building_points_in_0.001',
 'osm_culture_points_in_0.0075',
 'osm_city_closest_dist',
 'osm_crossing_points_in_0.0075',
 'osm_building_points_in_0.0075',
 'osm_historic_points_in_0.005',
 'osm_

In [34]:
TARGET = 'per_square_meter_price'
# признаки (или набор признаков), для которых применяем smoothed target encoding
CATEGORICAL_STE_FEATURES = ['region', 'city', 'realty_type']

# признаки, для которых применяем one hot encoding
CATEGORICAL_OHE_FEATURES = []

# численные признаки
NUM_FEATURES = ['lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
      'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
       'osm_historic_points_in_0.005', 'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
       'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500','total_square']



In [46]:
import typing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder



class GreatCategoriesEncoder:

    def __init__(self, numerical_features: typing.List[str], ohe_categorical_features: typing.List[str],
                 ste_categorical_features: typing.List[typing.Union[str, typing.List[str]]]):
        self.num_features = numerical_features
        self.ohe_cat_features = ohe_categorical_features
        self.ste_cat_features = ste_categorical_features

        self.preprocessor = ColumnTransformer(transformers=[
            ('num', StandardScaler(), self.num_features),
            ('ohe', OneHotEncoder(), self.ohe_cat_features),
            ('ste', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), self.ste_cat_features)])
        
    def use(self, df):
        return self.preprocessor.fit_transform(df)
        

In [47]:
gce = GreatCategoriesEncoder(numerical_features=NUM_FEATURES, ohe_categorical_features=CATEGORICAL_OHE_FEATURES,
                             ste_categorical_features=CATEGORICAL_STE_FEATURES)

In [48]:
df = pd.read_csv('train.csv')

D:\programmes\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [49]:
df_1 = gce.use(df)

In [51]:
df_1.shape

(279792, 70)

In [52]:
df.shape

(279792, 77)